<a target="_blank" href="https://colab.research.google.com/github/IngCarlaPezzone/tensorflow-1-public/blob/main/C1/W4/ungraded_labs/C1_W4_Lab_3_compacted_images_traducido.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Laboratorio no calificado: Efecto de las imágenes compactadas en el entrenamiento

En este cuaderno, verá cómo la reducción del tamaño objetivo de las imágenes del generador afectará a la arquitectura y al rendimiento de su modelo. Esta es una técnica útil en caso de que necesite acelerar su entrenamiento o ahorrar recursos de computación. Comencemos.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

# Ungraded Lab: Effect of Compacted Images in Training

In this notebook, you will see how reducing the target size of the generator images will affect the architecture and performance of your model. This is a useful technique in case you need to speed up your training or save compute resources. Let's begin!

**NOTA IMPORTANTE:** Este cuaderno está diseñado para ejecutarse como Colab. Ejecutarlo en su máquina local puede dar lugar a que algunos de los bloques de código arrojen errores.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

**IMPORTANT NOTE:** This notebook is designed to run as a Colab. Running it on your local machine might result in some of the code blocks throwing errors.

In [ ]:
# # Instale este paquete para utilizar la GPU de Colab para el entrenamiento
!apt install --allow-change-held-packages libcudnn8=8.4.1.50-1+cuda11.6

Como antes, comience a descargar los conjuntos de entrenamiento y validación:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

As before, start downloading the train and validation sets:

In [ ]:
# Descargar el conjunto de entrenamiento
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/horse-or-human.zip

In [ ]:
# Descargar el conjunto de validación
!wget https://storage.googleapis.com/tensorflow-1-public/course2/week3/validation-horse-or-human.zip

Entonces descomprímelos:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Then unzip them:

In [ ]:
import zipfile

# Descomprimir el conjunto de entrenamiento
local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

# Descomprimir conjunto de validación
local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

A continuación, defina los directorios que contienen las imágenes:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

Then define the directories containing the images:

In [ ]:
import os

# Directorio con fotos de caballos de entrenamiento
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directorio con fotos de humanos en entrenamiento
train_human_dir = os.path.join('./horse-or-human/humans')

# Directorio con imágenes de caballos de validación
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directorio con fotos de humanos de validación
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

Puede comprobar que los directorios no están vacíos y que el conjunto de trenes tiene más imágenes que el conjunto de validación:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can check that the directories are not empty and that the train set has more images than the validation set:

In [ ]:
train_horse_names = os.listdir(train_horse_dir)
print(f'TRAIN SET HORSES: {train_horse_names[:10]}')

train_human_names = os.listdir(train_human_dir)
print(f'TRAIN SET HUMANS: {train_human_names[:10]}')

validation_horse_hames = os.listdir(validation_horse_dir)
print(f'VAL SET HORSES: {validation_horse_hames[:10]}')

validation_human_names = os.listdir(validation_human_dir)
print(f'VAL SET HUMANS: {validation_human_names[:10]}')

In [ ]:
print(f'total training horse images: {len(os.listdir(train_horse_dir))}')
print(f'total training human images: {len(os.listdir(train_human_dir))}')
print(f'total validation horse images: {len(os.listdir(validation_horse_dir))}')
print(f'total validation human images: {len(os.listdir(validation_human_dir))}')

## Construir el modelo

El modelo seguirá la misma arquitectura que antes pero la diferencia clave está en el parámetro `input_shape` de la primera capa `Conv2D`. Dado que más tarde se compactarán las imágenes en el generador, es necesario especificar aquí el tamaño de imagen esperado. Así que en lugar de 300x300 como en los dos laboratorios anteriores, se especifica una matriz más pequeña de 150x150.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Build the Model

The model will follow the same architecture as before but they key difference is in the `input_shape` parameter of the first `Conv2D` layer. Since you will be compacting the images later in the generator, you need to specify the expected image size here. So instead of 300x300 as in the previous two labs, you specify a smaller 150x150 array.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # Obsérvese que la forma de entrada es el tamaño deseado de la imagen 150x150 con 3 bytes de color
    # Esta es la primera convolución
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # La segunda convolución
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # La tercer convolución
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
#     # La cuarta convolución (Puedes descomentar la 4ª y 5ª capa conv para ver el efecto)
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     # La quinta convolución
#     tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
    # Aplanar los resultados para alimentar una DNN
    tf.keras.layers.Flatten(),
    # Capa oculta de 512 neuronas
    tf.keras.layers.Dense(512, activation='relu'),
    # Sólo 1 neurona de salida. Contendrá un valor de 0-1 
    # donde 0 para una clase ('caballos') y 1 para la otra ('humanos')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

Se puede ver la diferencia con los modelos anteriores cuando se imprime el `model.summary()`. Como es de esperar, habrá menos entradas a la capa `Dense` al final del modelo en comparación con los laboratorios anteriores. Esto se debe a que ha utilizado el mismo número de capas de agrupación máxima en su modelo. Y como tiene una imagen más pequeña para empezar (150 x 150), entonces la salida después de todas las capas de agrupación también será más pequeña.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You can see the difference from previous models when you print the `model.summary()`. As expected, there will be less inputs to the `Dense` layer at the end of the model compared to the previous labs. This is because you used the same number of max pooling layers in your model. And since you have a smaller image to begin with (150 x 150), then the output after all the pooling layers will also be smaller.

In [ ]:
model.summary()

Utilizará los mismos ajustes para el entrenamiento:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

You will use the same settings for training:

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

### Preprocesamiento de datos

Ahora instanciaremos los generadores de datos. Como se mencionó anteriormente, se compactará la imagen especificando el parámetro `target_size`. Vea el sencillo cambio de abajo:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Data Preprocessing

Now you will instantiate the data generators. As mentioned before, you will be compacting the image by specifying the `target_size` parameter. See the simple change below:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#  Todas las imágenes serán reescaladas por 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  #  Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(150, 150),  # Todas las imágenes serán redimensionadas a 150x150
        batch_size=128,
        # Como se ha utilizado la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

# Flujo de imágenes de entrenamiento en lotes de 128 utilizando el generador train_datagen
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # Este es el directorio de origen para las imágenes de entrenamiento
        target_size=(150, 150),  # Todas las imágenes se redimensionarán a 150x150
        batch_size=32,
        # Como se ha utilizado la pérdida binaria_crossentropy, se necesitan etiquetas binarias
        class_mode='binary')

### Entrenamiento

Ahora está listo para entrenar y ver los resultados. Observa la rapidez con la que se entrena el modelo y las precisiones que obtienes en los conjuntos de entrenamiento y validación.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Training

Now you're ready to train and see the results. Note your observations about how fast the model trains and the accuracies you're getting in the train and validation sets.

In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=15,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

### Predicción del modelo

Como es habitual, también es una buena práctica intentar ejecutar su modelo sobre algunas imágenes seleccionadas. Compruebe si ha obtenido un rendimiento mejor, peor o igual que en el laboratorio anterior.

**Nota importante:** Debido a algunos problemas de compatibilidad, el siguiente bloque de código dará lugar a un error después de seleccionar las imágenes a cargar si está ejecutando este cuaderno como un `Colab` en el navegador `Safari`. Para todos los demás navegadores, continúe con el siguiente bloque de código e ignore el que le sigue.

Para los usuarios de Safari: por favor, comente u omita el bloque de código de abajo, descomente el siguiente bloque de código y ejecútelo.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Model Prediction

As usual, it is also good practice to try running your model over some handpicked images. See if you got better, worse, or the same performance as the previous lab.

**Important Note:** Due to some compatibility issues, the following code block will result in an error after you select the images(s) to upload if you are running this notebook as a `Colab` on the `Safari` browser. For all other browsers, continue with the next code block and ignore the next one after it.

_For Safari users: please comment out or skip the code block below, uncomment the next code block and run it._

In [ ]:
## BLOQUE DE CÓDIGO PARA NAVEGADORES QUE NO SEAN SAFARI
## USUARIOS DE SAFARI: POR FAVOR, OMITA ESTE BLOQUE Y EJECUTE EL SIGUIENTE EN SU LUGAR

import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
    # predicting images
    path = '/content/' + fn
    img = image.load_img(path, target_size=(150, 150))
    x = image.img_to_array(img)
    x /= 255
    x = np.expand_dims(x, axis=0)

    images = np.vstack([x])
    classes = model.predict(images, batch_size=10)
    print(classes[0])
    if classes[0]>0.5:
    print(fn + " is a human")
    else:
    print(fn + " is a horse")
 

Los usuarios de `Safari` tendrán que cargar las imágenes manualmente en su espacio de trabajo. Por favor, siga las instrucciones, descomente el bloque de código de abajo y ejecútelo.

Instrucciones para subir imágenes manualmente en un Colab:

1. Seleccione el icono de la `carpeta` en la `barra de menú` de la izquierda.
2. Haga clic en la `carpeta con una flecha apuntando hacia arriba` llamada `..`.
3. Haga clic en la `carpeta` llamada `tmp`.
4. Dentro de la carpeta `tmp`, `crea una nueva carpeta` llamada `imágenes`. Verás la opción `Nueva carpeta` haciendo clic en el botón de menú de los `3 puntos verticales` junto a la carpeta `tmp`.
5. Dentro de la nueva carpeta `imágenes`, sube una(s) imagen(es) de tu elección, preferiblemente de un caballo o de un humano. Arrastra y suelta la(s) imagen(es) encima de la carpeta `images`.
6. Descomente y ejecute el bloque de código que aparece a continuación. 

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

`Safari` users will need to upload the images(s) manually in their workspace. Please follow the instructions, uncomment the code block below and run it.

Instructions on how to upload image(s) manually in a Colab:

1. Select the `folder` icon on the left `menu bar`.
2. Click on the `folder with an arrow pointing upwards` named `..`
3. Click on the `folder` named `tmp`.
4. Inside of the `tmp` folder, `create a new folder` called `images`. You'll see the `New folder` option by clicking the `3 vertical dots` menu button next to the `tmp` folder.
5. Inside of the new `images` folder, upload an image(s) of your choice, preferably of either a horse or a human. Drag and drop the images(s) on top of the `images` folder.
6. Uncomment and run the code block below. 

In [ ]:
# # CODE BLOCK FOR SAFARI USERS

# import numpy as np
# from keras.preprocessing import image
# import os

# images = os.listdir("/tmp/images")

# print(images)

# for i in images:
#  print()
#  # predicting images
#  path = '/tmp/images/' + i
#  img = image.load_img(path, target_size=(150, 150))
#  x = image.img_to_array(img)
#  x /= 255
#  x = np.expand_dims(x, axis=0)

#  images = np.vstack([x])
#  classes = model.predict(images, batch_size=10)
#  print(classes[0])
#  if classes[0]>0.5:
#    print(i + " is a human")
#  else:
#    print(i + " is a horse")

### Visualización de las representaciones intermedias

También puede observar de nuevo las representaciones intermedias. Notará que la salida en la última capa de convolución es aún más abstracta porque contiene menos píxeles que antes.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

### Visualizing Intermediate Representations

You can also look again at the intermediate representations. You will notice that the output at the last convolution layer is even more abstract because it contains fewer pixels than before.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Definir un nuevo modelo que tomará una imagen como entrada, y emitirá
# representaciones intermedias para todas las capas del modelo anterior después de
# la primera.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = tf.keras.models.Model(inputs = model.input, outputs = successive_outputs)

# Prepara una imagen de entrada aleatoria del conjunto de entrenamiento.
horse_img_files = [os.path.join(train_horse_dir, f) for f in train_horse_names]
human_img_files = [os.path.join(train_human_dir, f) for f in train_human_names]
img_path = random.choice(horse_img_files + human_img_files)
img = load_img(img_path, target_size=(150, 150))  # esta es una imagen PIL
x = img_to_array(img)  # matriz Numpy con forma (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Matriz Numpy con forma (1, 150, 150, 3)

# Escala por  1/255
x /= 255

# Pasar la imagen por la red, obteniendo así todas
# representaciones intermedias de esta imagen.
successive_feature_maps = visualization_model.predict(x)

# Estos son los nombres de las capas, por lo que puede tenerlos como parte de la trama
layer_names = [layer.name for layer in model.layers[1:]]

# Visualizar las representaciones
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
    if len(feature_map.shape) == 4:

    # Sólo se hace esto para las capas conv / maxpool, no para las capas totalmente conectadas
    n_features = feature_map.shape[-1]  # number of features in feature map

    # El mapa de características tiene forma (1, size, size, n_features)
    size = feature_map.shape[1]

    # Mosaico de las imágenes en esta matriz
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
        x = feature_map[0, :, :, i]
        x -= x.mean()
        x /= x.std()
        x *= 64
        x += 128
        x = np.clip(x, 0, 255).astype('uint8')

        # Colocar cada filtro en esta gran cuadrícula horizontal
        display_grid[:, i * size : (i + 1) * size] = x

    # Mostrar la grilla
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

## Limpiar

Por favor, ejecute la siguiente celda para terminar el kernel y liberar recursos de memoria:

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Clean Up

Please run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)

## Wrap Up

En este laboratorio, has visto cómo la compactación de imágenes afectaba a tu modelo anterior. Esta es una técnica a tener en cuenta especialmente cuando aún estás en la fase de exploración de tus propios proyectos. Puedes ver si un modelo más pequeño se comporta igual de bien que un modelo grande y así tener un entrenamiento más rápido. También has visto lo fácil que es personalizar tus imágenes para este ajuste de tamaño simplemente cambiando un parámetro en la clase `ImageDataGenerator`.

<details><summary><font size="2" color="darkblue"><b> Texto Original </b></font></summary>

## Wrap Up

In this lab, you saw how compacting images affected your previous model. This is one technique to keep in mind especially when you are still in the exploratory phase of your own projects. You can see if a smaller model behaves just as well as a large model so you can have faster training. You also saw how easy it is to customize your images for this adjustment in size by simply changing a parameter in the `ImageDataGenerator` class.